In [ ]:
import dash
from dash import Dash
from dash.dependencies import Output, Input, State
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from numpy import loadtxt

In [ ]:
#load model
model = pickle.load(open("project1_model.sav", 'rb'))

#read data
df = pd.read_pickle("project1_data.pkl")

#get the mean and std
mean_std = loadtxt('mean_std.csv', delimiter=',')
X_train_mean = mean_std[0]
X_train_std = mean_std[1]

#get the training and testing data
X = df.drop(columns = ['y', 'label'])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.values
X_test  = X_test.values
y_train = y_train.values.astype('int')
y_test  = y_test.values.astype('int')

X_train = (X_train - X_train_mean) / X_train_std
X_test = (X_test - X_train_mean) / X_train_std

In [ ]:
y_pred = model.predict(X_test)
cf_matrix = confusion_matrix(y_test, y_pred)

precision  = cf_matrix[1,1] / sum(cf_matrix[:,1])
recall     = cf_matrix[1,1] / sum(cf_matrix[1,:])
f1_score  = 2*precision*recall / (precision + recall)

l_train, l_test = len(y_train), len(y_test)

df_class1 = df[df['label'] == 1]

In [ ]:
#calculate mean of all high-quality product features means as default value
features_mean = df_class1[['x1','x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13']].mean(axis=0)
features_mean = list(features_mean)
features_mean = [round(i,2) for i in features_mean]

In [ ]:
def Header(name, app):
    title = html.H2(name, style={"margin-top": 5})
    return dbc.Row([dbc.Col(title, md=9)])

def LabeledSelect(label, **kwargs):
    return dbc.FormGroup([dbc.Label(label), dbc.Select(**kwargs)])

def FeaturesCol(index,val):
    return dbc.Col(dbc.FormGroup([dbc.Label('Feature x{}'.format(index), size="md"), 
                                  dbc.Input(id="input-x{}".format(index),type="number",value=val)]), width=6) 

In [ ]:
#for test
f_dict={'x1': 'High-quality Product Feature X1 Distribution', 
        'x2': 'High-quality Product Feature X2 Distribution', 
        'x3': 'High-quality Product Feature X3 Distribution', 
        'x4': 'High-quality Product Feature X4 Distribution', 
        'x5': 'High-quality Product Feature X5 Distribution',
        'x6': 'High-quality Product Feature X6 Distribution',
        'x7': 'High-quality Product Feature X7 Distribution',
        'x8': 'High-quality Product Feature X8 Distribution', 
        'x9': 'High-quality Product Feature X9 Distribution', 
        'x10': 'High-quality Product Feature X10 Distribution', 
        'x11': 'High-quality Product Feature X11 Distribution', 
        'x12': 'High-quality Product Feature X12 Distribution', 
        'x13': 'High-quality Product Feature X13 Distribution'}

cards = [
    dbc.Card(
        [
            html.H2(f"{f1_score*100:.2f}%", className="card-title"),
            html.P("Model Test F1 Score", className="card-text"),
        ],
        body=True,
        color="light",
    ),
    dbc.Card(
        [
            html.H2(f"{precision*100:.2f}%", className="card-title"),
            html.P("Model Test Precision", className="card-text"),
        ],
        body=True,
        color="dark",
        inverse=True,
    ),
    dbc.Card(
        [
            html.H2(f"{l_train} / {l_test}", className="card-title"),
            html.P("Train / Test Split", className="card-text"),
        ],
        body=True,
        color="primary",
        inverse=True,
    ),
]

graphs = [
    [
        LabeledSelect(
            id="selected-features",
            options=[{"label": v, "value": k} for k, v in f_dict.items()],
            label="Features",
            value='x1'
        ),
        dcc.Graph(id='graph_feature'),
    ],
    [        
        dbc.Row([FeaturesCol(1,features_mean[0]), FeaturesCol(2,features_mean[1])]), 
        dbc.Row([FeaturesCol(3,features_mean[2]), FeaturesCol(4,features_mean[3])]), 
        dbc.Row([FeaturesCol(5,features_mean[4]), FeaturesCol(6,features_mean[5])]), 
        dbc.Row([FeaturesCol(7,features_mean[6]), FeaturesCol(8,features_mean[7])]), 
        dbc.Row([FeaturesCol(9,features_mean[8]), FeaturesCol(10,features_mean[9])]), 
        dbc.Row([FeaturesCol(11,features_mean[10]), FeaturesCol(12,features_mean[11])]), 
        dbc.Row([FeaturesCol(13,features_mean[12])])
    ],

]

results = [
    [
        dbc.Button("Prediction", id="pred_submit", n_clicks=0,outline=True, color="primary", className="me-1")
    ],
    [
        html.Span(id="Prediction Result", style={'font-style': 'italic', 'font-weight': 'bold'})
    ]
]

In [ ]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

@app.callback(
    Output("graph_feature", "figure"),
    Output("Prediction Result", "children"),
    Input("selected-features", "value"),
    Input('pred_submit', 'n_clicks'),
    State("input-x1", "value"),
    State("input-x2", "value"),
    State("input-x3", "value"),
    State("input-x4", "value"),
    State("input-x5", "value"),
    State("input-x6", "value"),
    State("input-x7", "value"),
    State("input-x8", "value"),
    State("input-x9", "value"),
    State("input-x10", "value"),
    State("input-x11", "value"),
    State("input-x12", "value"),
    State("input-x13", "value"),
)
def update_figures(sele_f, _,x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13):
    #Fileter based on the selected features
    fig = px.histogram(df_class1,
                       x = sele_f,
                       marginal="box",
                       color_discrete_sequence=['teal'])
    input_data = [[x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13]]
    input_data = (input_data - X_train_mean) / X_train_std
    output_pred = model.predict(input_data)[0]
    
    rslt = "Based on the input data, we will get a {} product.".format("High-Quality" if output_pred == 1 else "Low-Quality")
    return fig, rslt
    

In [ ]:
app.layout = dbc.Container(
    [
        Header("Project1 Production Quality Prediction", app),
        html.Hr(),
        dbc.Row([dbc.Col(card) for card in cards]),
        html.Br(),
        dbc.Row([dbc.Col(graph) for graph in graphs]),
        html.Br(),
        dbc.Row([dbc.Col(results[0], width=1.5), dbc.Col(results[1], width=9)])
    ],
    fluid=False,
)

if __name__ == "__main__":
    app.run_server(debug=False)